## Bibliotecas

In [1]:
import pandas as pd
import numpy as np
import nltk
import string
from sklearn.dummy import DummyClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
from gensim.models import KeyedVectors

## Carregando os dados

In [2]:
artigo_treino = pd.read_csv('Dados/treino.csv')
artigo_teste = pd.read_csv('Dados/teste.csv')

## Visualizando os dados

In [3]:
# Visualizando os dados de treino.
artigo_treino.sample(5)

,title,text,date,category,subcategory,link
2454,Mostra de teatro em SP confirma participação d...,A MITsp (Mostra Internacional de Teatro de São...,2015-02-18,ilustrada,NaN,http://www1.folha.uol.com.br/ilustrada/2015/02...
38032,Possível derrubada de veto à lei do ISS nesta ...,A possibilidade de que seja votada nesta seman...,2017-05-16,colunas,mercadoaberto,http://www1.folha.uol.com.br/colunas/mercadoab...
60963,Justiça decreta prisão de mais três suspeitos ...,A 1ª Vara Judicial de São Jerônimo (a 70 km de...,2016-07-22,cotidiano,NaN,http://www1.folha.uol.com.br/cotidiano/2016/07...
87383,"Prédios históricos, e um trema, têm significad...",Para as pessoas que pertencem a determinado lu...,2015-04-25,mundo,NaN,http://www1.folha.uol.com.br/mundo/2015/04/162...
57335,Senador do PSDB diz que reforma da Previdência...,O senador Antônio Anastasia (PSDB-MG) afirmou ...,2017-04-24,mercado,NaN,http://www1.folha.uol.com.br/mercado/2017/04/1...


In [4]:
# Visualizando os dados de teste.
artigo_teste.sample(5)

,title,text,date,category,subcategory,link
17055,Pai de Neymar isenta o atacante durante depoim...,O atacante Neymar Jr. foi isentado de culpa po...,2016-02-02,esporte,NaN,http://www1.folha.uol.com.br/esporte/2016/02/1...
19089,Aplicativo ajuda a combater violência contra a...,"Com apenas quatro toques, a polícia, a Justiça...",2016-01-26,cotidiano,NaN,http://www1.folha.uol.com.br/cotidiano/2016/01...
8692,Sucesso do prefeito depende da burocracia,Escolher um bom candidato a prefeito é o prime...,2016-09-29,colunas,fernando-abrucio,http://www1.folha.uol.com.br/colunas/fernando-...
20009,A vitória da melancia quadrada,"RIO DE JANEIRO - Há quase dez anos, em agosto ...",2017-06-26,colunas,ruycastro,http://www1.folha.uol.com.br/colunas/ruycastro...
614,Recado da Otan à Rússia é que 'Báltico não é o...,"Dois B-52, gigantescos bombardeiros criados no...",2015-06-24,mundo,NaN,http://www1.folha.uol.com.br/mundo/2015/06/164...


## Lendo cbow.txt

In [5]:
# Lendo o arquivo de vetores pré-treinados.
with open (r'.\Dados\cbow_s300.txt', 'r', encoding='utf-8') as f:
    for linha in range(10):
        print(next(f))

929606 300

</s> 0.001334 0.001473 -0.001277 -0.001093 0.000456 0.001007 0.000314 0.000070 -0.001201 0.000739 -0.001452 0.000417 -0.000250 -0.000319 -0.001105 -0.000627 0.000860 0.001008 0.000990 0.000532 0.000515 -0.001268 -0.001365 0.001657 0.001267 0.001030 -0.000201 0.001339 -0.000165 0.000245 -0.000050 -0.000994 0.000437 -0.000446 -0.001275 0.001585 0.001460 -0.000365 -0.000075 0.000170 -0.001213 -0.001336 0.001518 0.000021 -0.000861 -0.001014 -0.001025 0.000566 0.000067 0.000444 -0.001405 -0.001269 -0.000043 0.000381 0.000850 -0.001057 0.001360 0.000309 0.000373 -0.000203 0.000995 -0.000763 -0.000699 0.000719 -0.000251 0.000344 0.000602 -0.001363 -0.000661 0.000971 0.001411 0.001311 -0.001016 -0.000703 -0.000303 0.000667 -0.001263 0.000999 0.000929 -0.000533 -0.000517 -0.000746 0.001410 0.001304 -0.000393 0.001405 0.001607 0.000605 0.001661 -0.001037 -0.000511 -0.000702 -0.000969 0.000938 0.000526 0.000142 -0.000731 0.000508 0.001020 0.000065 0.000760 -0.001651 0.001438 0.000160 

In [6]:
modelo = KeyedVectors.load_word2vec_format(r'.\Dados\cbow_s300.txt')

## Vetorização de texto

### Tokenização

In [7]:
# Criando uma função para tokenizar o texto.
def tokenizador(texto):
    # deixando o texto em minúsculo
    texto = texto.lower()

    # criando uma lista vazia para armazenar os tokens.
    lista_alfanumerico = []

    # para cada token no texto, se o token não estiver na lista de pontuações, adicione-o à lista de tokens.
    for token_valido in nltk.word_tokenize(texto):
        if token_valido in string.punctuation: continue
        lista_alfanumerico.append(token_valido)      

    return lista_alfanumerico

In [8]:
# Criando uma função para vetorizar o texto.
def combinacao_de_vetores_por_soma(palavras_numeros):
    # definindo um vetor de tamanho 300 com valores 0.
    vetor_resultante = np.zeros(300)

    # para cada palavra-numero no texto, soma o vetor da palavra ao vetor_resultante.
    for pn in palavras_numeros:
        try:
            vetor_resultante += modelo.get_vector(pn)    
        except KeyError:
            if pn.isnumeric():
                pn = '0'*len(pn)
                vetor_resultante += modelo.get_vector(pn)
            else:
                vetor_resultante += modelo.get_vector('unknown')
    return vetor_resultante

## Vetorizando os dados

In [9]:
# Criando uma função para vetorizar os dados de treino e teste.
def matriz_vetores(textos):
    matriz = np.zeros((len(textos), 300))

    for i in range(len(textos)):
        palavras_numeros = tokenizador(textos.iloc[i])
        matriz[i] = combinacao_de_vetores_por_soma(palavras_numeros)

    return matriz

# Vetorizando os dados de treino e teste.
matriz_vetores_treino = matriz_vetores(artigo_treino.title)
matriz_vetores_teste = matriz_vetores(artigo_teste.title)

print(matriz_vetores_treino.shape)
print(matriz_vetores_teste.shape)

(90000, 300)
(20513, 300)


## Criando o modelo

### Criando modelo - Regressão Logística

In [10]:
# Instanciando o modelo de Regressão Logística.
LR = LogisticRegression(max_iter=550)

# Treinando o modelo.
LR.fit(matriz_vetores_treino, artigo_treino.category)

# Verificando a acurácia do modelo.
print(f'Acurácia: {LR.score(matriz_vetores_teste, artigo_teste.category)}')

# Verificando a quantidade de iteracões.
print(f'Quantidade de iterações: {LR.n_iter_}')

Acurácia: 0.8163603568468776
Quantidade de iterações: [540]


### Classification Report

In [11]:
# Fazendo as previsões.
label_prevista = LR.predict(matriz_vetores_teste)

# Verificando o relatório de classificação.
CR = classification_report(artigo_teste.category, label_prevista)

# Imprimindo o relatório de classificação.
print(CR)

              precision    recall  f1-score   support

     colunas       0.87      0.72      0.79      6103
   cotidiano       0.64      0.82      0.72      1698
     esporte       0.94      0.90      0.92      4663
   ilustrada       0.15      0.90      0.26       131
     mercado       0.85      0.83      0.84      5867
       mundo       0.78      0.87      0.83      2051

    accuracy                           0.82     20513
   macro avg       0.70      0.84      0.72     20513
weighted avg       0.85      0.82      0.83     20513



### Criando modelo - Dummy Classifier

In [12]:
# Instanciando o modelo DummyClassifier.
DC = DummyClassifier()

# Treinando o modelo.
DC.fit(matriz_vetores_treino, artigo_treino.category)

# Faça as previsões.
label_prevista_dc = DC.predict(matriz_vetores_teste)

# Verificando o relatório de classificação.
CR_dummy = classification_report(artigo_teste.category, label_prevista_dc)

# Imprimindo o relatório de classificação.
print(CR_dummy)



c:\Users\natsu\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


              precision    recall  f1-score   support

     colunas       0.30      1.00      0.46      6103
   cotidiano       0.00      0.00      0.00      1698
     esporte       0.00      0.00      0.00      4663
   ilustrada       0.00      0.00      0.00       131
     mercado       0.00      0.00      0.00      5867
       mundo       0.00      0.00      0.00      2051

    accuracy                           0.30     20513
   macro avg       0.05      0.17      0.08     20513
weighted avg       0.09      0.30      0.14     20513



c:\Users\natsu\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\natsu\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


## Classificando com Skip-gram

### Lendo skipgram.txt

In [13]:
# Lendo o arquivo de vetores pré-treinados.
with open (r'.\Dados\skip_s300.txt', 'r', encoding='utf-8') as f:
    for linha in range(10):
        print(next(f))

929606 300

</s> 0.001334 0.001473 -0.001277 -0.001093 0.000456 0.001007 0.000314 0.000070 -0.001201 0.000739 -0.001452 0.000417 -0.000250 -0.000319 -0.001105 -0.000627 0.000860 0.001008 0.000990 0.000532 0.000515 -0.001268 -0.001365 0.001657 0.001267 0.001030 -0.000201 0.001339 -0.000165 0.000245 -0.000050 -0.000994 0.000437 -0.000446 -0.001275 0.001585 0.001460 -0.000365 -0.000075 0.000170 -0.001213 -0.001336 0.001518 0.000021 -0.000861 -0.001014 -0.001025 0.000566 0.000067 0.000444 -0.001405 -0.001269 -0.000043 0.000381 0.000850 -0.001057 0.001360 0.000309 0.000373 -0.000203 0.000995 -0.000763 -0.000699 0.000719 -0.000251 0.000344 0.000602 -0.001363 -0.000661 0.000971 0.001411 0.001311 -0.001016 -0.000703 -0.000303 0.000667 -0.001263 0.000999 0.000929 -0.000533 -0.000517 -0.000746 0.001410 0.001304 -0.000393 0.001405 0.001607 0.000605 0.001661 -0.001037 -0.000511 -0.000702 -0.000969 0.000938 0.000526 0.000142 -0.000731 0.000508 0.001020 0.000065 0.000760 -0.001651 0.001438 0.000160 

In [14]:
modelo_skipgram = KeyedVectors.load_word2vec_format(r'.\Dados\skip_s300.txt')

### Vetorizando os dados

In [15]:
# Criando uma função para vetorizar o texto.
def combinacao_de_vetores_por_soma_skipgram(palavras_numeros):
    # definindo um vetor de tamanho 300 com valores 0.
    vetor_resultante = np.zeros(300)

    # para cada palavra-numero no texto, soma o vetor da palavra ao vetor_resultante.
    for pn in palavras_numeros:
        try:
            vetor_resultante += modelo_skipgram.get_vector(pn)    
        except KeyError:
            if pn.isnumeric():
                pn = '0'*len(pn)
                vetor_resultante += modelo_skipgram.get_vector(pn)
            else:
                vetor_resultante += modelo_skipgram.get_vector('unknown')
    return vetor_resultante

In [16]:
# Criando uma função para vetorizar os dados de treino e teste.
def matriz_vetores_skipgram(textos):
    matriz = np.zeros((len(textos), 300))

    for i in range(len(textos)):
        palavras_numeros = tokenizador(textos.iloc[i])
        matriz[i] = combinacao_de_vetores_por_soma_skipgram(palavras_numeros)

    return matriz

# Vetorizando os dados de treino e teste.
matriz_vetores_treino_skipgram = matriz_vetores_skipgram(artigo_treino.title)
matriz_vetores_teste_skipgram = matriz_vetores_skipgram(artigo_teste.title)

print(matriz_vetores_treino_skipgram.shape)
print(matriz_vetores_teste_skipgram.shape)

(90000, 300)
(20513, 300)


### Criando modelo - Regressão Logística

In [17]:
# Instanciando o modelo de Regressão Logística.
LR_skipgram = LogisticRegression(max_iter=750)

# Treinando o modelo.
LR_skipgram.fit(matriz_vetores_treino_skipgram, artigo_treino.category)

# Realizando as previsões.
label_prevista_skipgram = LR_skipgram.predict(matriz_vetores_teste_skipgram)

# Verificando o relatório de classificação.
CR_skipgram = classification_report(artigo_teste.category, label_prevista_skipgram)

# Imprimindo o relatório de classificação.
print(CR_skipgram)

# Verificando a quantidade de iteracões.
print(f'Quantidade de iterações: {LR_skipgram.n_iter_}')

              precision    recall  f1-score   support

     colunas       0.87      0.72      0.79      6103
   cotidiano       0.64      0.82      0.72      1698
     esporte       0.93      0.90      0.92      4663
   ilustrada       0.15      0.89      0.26       131
     mercado       0.85      0.83      0.84      5867
       mundo       0.79      0.87      0.83      2051

    accuracy                           0.82     20513
   macro avg       0.71      0.84      0.73     20513
weighted avg       0.85      0.82      0.83     20513

Quantidade de iterações: [615]
